In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [2]:
import sys
sys.path.append("..")

# MAKE SURE TO SET-UP PATH -> use local to run with demo data; use azure to run with complete dataset (access required)
# Select where to run notebook: "azure" or "local"
my_run = "local"

if my_run == "azure":
    import config_azure as cf
    running_demo = False
elif my_run == "local":
    import config as cf
    running_demo = True


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE

import pandas as pd


## Notebook overview
Goal: manually check some of the classes. especially docs that have very low token count.
- Starting point: txtfiles_notcleaned.pkl
- Ending point: txtfiles.pkl -> only including 'good' documents

We found that there are documents with very low token count. Thus we further checked for each class how clean the dataset was. This led to the complete removal of three classes and the removal of some documents based on their length. 

In this notebook:
1. We will first display the token distribution to highlight the imbalance. 
2. Then, we will provide the code to remove the faulty documents.
3. After, we also remove duplicate documents. 
4. Then, we remove unneccary columns. We don't need the tokens, token_count, clean_tokens, clean_tokens_count columns anymore, those were used for analysis of the raw data.
5. Finally, we will explain the reasons for the removal of the faulty documents, if applicable, for each class. The explanation is put at the end of the notebook since it takes up a lot of cells. 

Previous notebook: load_txt.ipynb

Next notebook: duplicates.ipynb

### 1. Check document length
We load in the unclean dataset and look into the document length. We can see that some documents are unusually short. 

In [3]:
# load data
import ast
import pandas as pd

txtfiles = pd.read_pickle(f"{cf.output_path}/txtfiles_notcleaned.pkl")
df = txtfiles.copy()

display(txtfiles.groupby('label')['token_count'].describe())

# clean tokens -> stopwords, punctuation etc. removed. very simple cleaning of tokens applied
display(txtfiles.groupby('label')['clean_tokens_count'].describe())


,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
actualiteit,10.0,535.0,470.433370,108.0,233.75,376.0,633.00,1606.0
agenda,10.0,527.1,626.263957,111.0,173.75,240.0,687.75,2157.0
besluit,10.0,2145.7,3939.064160,112.0,246.75,993.0,1873.50,13155.0
brief,10.0,2064.0,2160.666307,287.0,694.00,1136.5,2378.00,6906.0
factsheet,10.0,1642.7,953.800241,679.0,823.75,1481.0,2035.00,3388.0
motie,10.0,268.5,72.873178,190.0,219.25,248.5,293.50,404.0
onderzoeksrapport,10.0,14672.2,8575.697416,880.0,9840.25,13164.0,20732.75,28872.0
raadsadres,10.0,842.0,461.315269,183.0,541.75,750.5,1262.00,1526.0
raadsnotulen,10.0,32888.6,10886.157226,18848.0,24880.25,31943.5,36826.25,53547.0


,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
actualiteit,10.0,272.0,236.156257,51.0,111.75,190.0,362.00,790.0
agenda,10.0,293.4,319.327488,72.0,107.50,155.0,385.50,1115.0
besluit,10.0,1071.2,1951.986612,62.0,132.75,497.0,935.25,6527.0
brief,10.0,1010.6,1062.573313,154.0,339.00,555.0,1159.00,3397.0
factsheet,10.0,1006.9,502.997338,467.0,560.50,916.0,1438.25,1778.0
motie,10.0,136.4,37.107052,99.0,111.00,122.5,146.75,203.0
onderzoeksrapport,10.0,7659.5,4403.828745,504.0,5192.75,6929.5,10964.75,14792.0
raadsadres,10.0,410.3,223.746409,79.0,259.50,352.0,602.50,783.0
raadsnotulen,10.0,14854.4,5198.432674,8394.0,11067.00,13942.5,16973.75,24926.0


### 2. Remove the faulty documents.
Below each class is checked, and explained why these documents are removed.

The removal includes:
- classes: 'Adviesaanvraag', 'Amendement', 'Begroting'
- some documents that are gibberish


In [4]:
print("Original amount of docs:", len(txtfiles))
# remove Adviesaanvraag, Amendement and Begroting. These classes include docs from other classes.
txtfiles = txtfiles.loc[~txtfiles['label'].isin(['Adviesaanvraag', 'Amendement', 'Begroting'])]

# remove docs that are too short (only include gibberish)
subdf = txtfiles.loc[txtfiles['label']== 'Schriftelijke Vragen']
threshold = subdf['clean_tokens_count'].quantile(0.01)
subdf = subdf.loc[txtfiles['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
txtfiles = txtfiles.loc[~txtfiles['id'].isin(subdf['id'])]

subdf = txtfiles.loc[(txtfiles['label']== 'Brief') & (txtfiles['clean_tokens_count'] < 75)]
txtfiles = txtfiles.loc[~txtfiles['id'].isin(subdf['id'])]

subdf = txtfiles.loc[(txtfiles['label']== 'Raadsadres') & (txtfiles['clean_tokens_count'] < 25)]
txtfiles = txtfiles.loc[~txtfiles['id'].isin(subdf['id'])]

# rename verslag column
txtfiles.loc[txtfiles['label']=='Verslag','label'] = 'Raadsnotulen'


print("Amount of docs after removal:", len(txtfiles))

Original amount of docs: 110
Amount of docs after removal: 110


### 3. Remove duplicates

goal: check data for duplicates

- Use md5 hashing to remove duplicates. 
- Data is re-split into subsets, using 2-split (train and test), 4-split (train, test, val and dev) and balance-split (train, test and val; used for research)

In [5]:
import hashlib
import sys

# load function to split data into subsets (train,test,val and dev)
sys.path.append('../src/') 
from data_split import save_split, save_balanced_split

def calculate_md5(file_path):
    """Calculate the MD5 hash of a file."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

"""Calculate MD5 hash for each file path in a DataFrame."""
def drop_duplicates(df, file_path_column):
    df['md5_hash'] = df[file_path_column].apply(calculate_md5)

    # remove rows with duplicate md5_hash
    clean_df = df.drop_duplicates(subset=['md5_hash'])
    print(f"{len(df)-len(clean_df)} docs removed. New total: {len(clean_df)} docs.")
    return clean_df

""" After removing duplicates and faulty documents, data needs to be resplit """
def redo_datasplit(df):
    df = save_split(df)
    df = save_balanced_split(df, demo=running_demo)
    return df

hash_df = drop_duplicates(txtfiles, 'path')
cleaned_df = redo_datasplit(hash_df)


6 docs removed. New total: 104 docs.


### 4. Remove unnessecary columns & save cleaned df

In [6]:
print(f"Columns before removing: {list(cleaned_df.columns)}")
cleaned_df = cleaned_df.drop(columns=['tokens', 'token_count', 'clean_tokens', 'clean_tokens_count'])
print(f"Columns after removing: {list(cleaned_df.columns)}")


Columns before removing: ['label', 'path', 'id', '4split', '2split', 'balanced_split', 'text', 'tokens', 'token_count', 'clean_tokens', 'clean_tokens_count', 'pdf_path', 'num_pages', 'md5_hash']
Columns after removing: ['label', 'path', 'id', '4split', '2split', 'balanced_split', 'text', 'pdf_path', 'num_pages', 'md5_hash']


In [7]:
cleaned_df.to_pickle(f"{cf.output_path}/txtfiles.pkl")

### 5. Explanation: Check each class

NOTE: this analysis is run on 'df', which is the uncleaned dataframe.

**Actualiteit**

After checking the lowest 1 percent of number of tokens (10 documents which have less than 58 tokens) were checked using the PDFs. These documents are just very short, thus not a mistake of the txt extraction. 

In [36]:
actualiteit = df.loc[df['label']=='Actualiteit']
threshold = actualiteit['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = actualiteit[actualiteit['clean_tokens_count'] < threshold]
# actualiteit['clean_tokens_count'].describe()


**Adviesaanvraag**

Txt extraction from PDFs went well, except for image. The images result in gibberish in the data. Additionally, it does not seem like there are only adviesaanvragen in there. 

In [37]:
adviesaanvraag = df.loc[df['label']=='Adviesaanvraag']
threshold = adviesaanvraag['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = adviesaanvraag[adviesaanvraag['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# adviesaanvraag['clean_tokens_count'].describe()


**Agenda**

Agenda's can just be very short. Txt extraction went well.

In [38]:
subdf = df.loc[df['label']=='Agenda']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Amendement**

Not only amendement docs are included. txt extraction went well.

In [39]:
subdf = df.loc[df['label']=='Amendement']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])

# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Begroting**

It does not seem like there are only begrotingen. txt went well. except for tables (file:///C:/Users/femke/Documents/MasterThesis/discardfiles/1543430.pdf)

In [40]:
subdf = df.loc[df['label']=='Begroting']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Besluit**

Looks good

In [41]:
subdf = df.loc[df['label']=='Besluit']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Brief**

Remove docs with less than 75 tokens. These are some weird poster. The rest looks good.

In [42]:
subdf = df.loc[df['label']=='Brief']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Factsheets**

Lots of posters. Lots of images included

In [43]:
subdf = df.loc[df['label']=='Factsheets']
threshold = subdf['clean_tokens_count'].quantile(0.1)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Motie**

I cant see the original PDFs. Looks good tho.

In [44]:
subdf = df.loc[df['label']=='Motie']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Onderzoeksrapport**   
   
Include posters, presentations? (/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2019-gemeenteraad/5g-technische-sessie.pdf.ocr
)


Includes images.

In [45]:
subdf = df.loc[df['label']=='Onderzoeksrapport']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Raadsadres**

remove less than 25 tokens. looks good.

In [46]:
subdf = df.loc[df['label']=='Raadsadres']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Schriftelijke Vragen**

Cant open PDFs. Looks like a mess. Middle looks much better. Should remove lowest 0.01 cleaned_tokens_count

In [47]:
subdf = df.loc[df['label']== 'Schriftelijke Vragen']
threshold = subdf['clean_tokens_count'].quantile(0.01)
# threshold2 = subdf['clean_tokens_count'].quantile(0.02)

lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# lower_5_percentile_rows = subdf[(subdf['clean_tokens_count'] < threshold2) & (subdf['clean_tokens_count']>threshold)].sort_values(by=['clean_tokens_count'])

# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()



**Termijnagenda**

Does include many tables.
Should be combined with agenda.

In [48]:
subdf = df.loc[df['label']=='Termijnagenda']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Verslag**

Should rename into raadsnotulen. looks good

In [51]:
subdf = df.loc[df['label']=='Verslag']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


**Voordracht**

Looks good

In [54]:
subdf = df.loc[df['label']=='Voordracht']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()
